In [1]:
import pandas as pd
import numpy as np
import swifter

c:\Users\phku0\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_parquet('merged_data/news_with_features_merged.parquet')
print(df.shape)
df.head()

(1904665, 21)


,title_preprocessed,summary_preprocessed,title,summary,time_published,source,title_sentiment_finbert,summary_sentiment_finbert,title_sentiment_vader,summary_sentiment_vader,...,topic_announcements,topic_market_movements,topic_legal_actions,topic_investment_sentiment,topic_earnings,topic_analyst_ratings,topic_macro_econ,topic_research_insights,company_names,stock_tickers
newsID,,,,,,,,,,,,,,,,,,,,,
1,"[oil, price, could, determine, market, react, ...","[heavy, new, round, sanction, russia, ally, li...",Oil prices could determine how markets react t...,The heavy new round of sanctions on Russia by ...,2022-03-01 00:02:02,CNBC,"{'label': 'Negative', 'score': 0.63124018907547}","{'label': 'Negative', 'score': 0.6197944283485...",0.0000,0.0000,...,False,False,False,False,False,False,True,False,[],[]
2,"[zoom, provide, disappointing, revenue, foreca...","[zoom, revenue, growth, continue, slow, busine...",Zoom provides disappointing revenue forecast f...,Zoom's revenue growth is continuing to slow af...,2022-03-01 00:15:56,CNBC,"{'label': 'Negative', 'score': 0.9999985694885...","{'label': 'Negative', 'score': 0.9999996423721...",-0.4939,0.3818,...,False,False,False,False,False,False,True,False,[Zoom],[]
3,"[wall, street, rally, west, hit, russia, new, ...","[rise, end, four, day, slide, amid, worry, esc...",Wall Street rallies as West hits Russia with n...,"The SP 500 rose more than 1%, ending a four-da...",2022-03-01 00:46:51,Money Control,"{'label': 'Neutral', 'score': 0.9479693174362183}","{'label': 'Positive', 'score': 0.9999978542327...",0.0000,-0.5106,...,False,False,False,False,True,False,False,False,[],[]
4,"[weak, manufacturing, drag, gdp, growth, oil, ...","[india, economy, grow, three, month, end, dece...","Weak manufacturing drags down Q3 GDP growth, o...",India's economy grew 5.4% in the three months ...,2022-03-01 02:23:00,Economic Times,"{'label': 'Negative', 'score': 0.9999984502792...","{'label': 'Negative', 'score': 0.9954470992088...",0.1531,-0.2960,...,False,False,False,False,False,False,True,False,[],[]
5,"[singapore, bank, halt, lend, russian, good, j...","[singapore, big, bank, restrict, trade, financ...",Singapore banks halt lending for Russian goods...,Singapore's biggest banks are restricting trad...,2022-03-01 02:30:56,South China Morning Post,"{'label': 'Negative', 'score': 0.9858503937721...","{'label': 'Negative', 'score': 0.9719559550285...",-0.1027,-0.4215,...,False,False,False,False,False,False,False,True,[],[]


In [3]:
df = df[['title_preprocessed', 'summary_preprocessed', 'stock_tickers']]
df.head()

,title_preprocessed,summary_preprocessed,stock_tickers
newsID,,,
1,"[oil, price, could, determine, market, react, ...","[heavy, new, round, sanction, russia, ally, li...",[]
2,"[zoom, provide, disappointing, revenue, foreca...","[zoom, revenue, growth, continue, slow, busine...",[]
3,"[wall, street, rally, west, hit, russia, new, ...","[rise, end, four, day, slide, amid, worry, esc...",[]
4,"[weak, manufacturing, drag, gdp, growth, oil, ...","[india, economy, grow, three, month, end, dece...",[]
5,"[singapore, bank, halt, lend, russian, good, j...","[singapore, big, bank, restrict, trade, financ...",[]


In [4]:
# join title and summary tokens
df['text_tokens'] = df.swifter.apply(lambda row: list(row['title_preprocessed']) + list(row['summary_preprocessed']), axis=1)
df.drop(columns=['title_preprocessed', 'summary_preprocessed'], inplace=True)
df.head()

Pandas Apply: 100%|██████████| 1904665/1904665 [00:14<00:00, 130111.80it/s]


,stock_tickers,text_tokens
newsID,,
1,[],"[oil, price, could, determine, market, react, ..."
2,[],"[zoom, provide, disappointing, revenue, foreca..."
3,[],"[wall, street, rally, west, hit, russia, new, ..."
4,[],"[weak, manufacturing, drag, gdp, growth, oil, ..."
5,[],"[singapore, bank, halt, lend, russian, good, j..."


### sector keywords matching

In [5]:
# Define sector keywords
sector_keywords = {
    "Energy": [
        "oil", "gas", "petroleum", "drilling", "refinery", "refining", "fossil fuel", "coal" 
        "crude", "drilling", "opec", "energy", "renewables", "solar", "wind", "hydrocarbon"
    ],
    "Materials": [
        "chemicals", "metals", "mining", "steel", "aluminum", "copper", "gold", "silver", "commodity", 
        "paper", "forestry", "construction materials", "packaging", "fertilizer", "minerals", "cement"
    ],
    "Industrials": [
        "manufacturing", "aerospace", "defense", "machinery", "construction", "engineering", "airlines", 
        "transportation", "logistics", "shipping", "railroad", "industrial", "waste management", 
        "infrastructure", "electrical", "distribution"
    ],
    "Consumer Discretionary": [
        "retail", "apparel", "luxury", "automobile", "vehicle", "car", "hotels", "restaurant", "leisure", 
        "travel", "entertainment", "media", "e-commerce", "consumer spending", "durables", "homebuilding", 
        "furniture", "tourism", "discretionary"
    ],
    "Consumer Staples": [
        "food", "beverage", "tobacco", "household", "personal products", "grocery", "supermarket", 
        "staples", "consumer packaged goods", "cpg", "fmcg", "dairy", "meat", "agriculture"
    ],
    "Health Care": [
        "pharmaceutical", "biotech", "medical", "healthcare", "hospital", "drug", "medicine",
        "insurance", "health", "therapy", "diagnostics", "dental", "clinical", "patient", "physician", 
        "therapeutic", "vaccine"
    ],
    "Financials": [
        "bank", "insurance", "investment", "asset management", "broker", "credit", "loan", "mortgage", 
        "capital market", "financial", "wealth management", "payment", "private equity", "venture capital", 
        "fintech", "banking", "real estate investment"
    ],
    "Information Technology": [
        "software", "hardware", "semiconductor", "technology", "computing", "internet", "cloud", "data", 
        "ai", "artificial intelligence", "machine learning", "cybersecurity", "tech", "digital", "it", 
        "information technology", "computer", "telecom", "electronics", "fintech", "blockchain"
    ],
    "Communication Services": [
        "telecommunication", "telecom", "wireless", "broadband", "internet service", "media", "advertising", 
        "publishing", "entertainment", "social media", "streaming", "cable", "network", "broadcast", 
        "communication"
    ],
    "Utilities": [
        "electricity", "gas utility", "water", "power", "utility", "electric", "renewable energy", 
        "grid", "nuclear", "generation", "transmission", "distribution", "energy company"
    ],
    "Real Estate": [
        "reit", "real estate", "property", "commercial real estate", "residential", "housing", "apartment", 
        "office", "retail space", "land", "development", "leasing", "rental", "commercial property"
    ]
}

In [6]:
def get_sector_matches(tokens):
  matched_sectors = set()
  for sector, keywords in sector_keywords.items():
    if any(token in keywords for token in tokens):
      matched_sectors.add(sector)
  return matched_sectors

df['sectors'] = df['text_tokens'].swifter.apply(get_sector_matches)

Pandas Apply: 100%|██████████| 1904665/1904665 [01:13<00:00, 25826.68it/s]


In [7]:
df.head()

,stock_tickers,text_tokens,sectors
newsID,,,
1,[],"[oil, price, could, determine, market, react, ...",{Energy}
2,[],"[zoom, provide, disappointing, revenue, foreca...",{}
3,[],"[wall, street, rally, west, hit, russia, new, ...",{}
4,[],"[weak, manufacturing, drag, gdp, growth, oil, ...","{Industrials, Energy}"
5,[],"[singapore, bank, halt, lend, russian, good, j...","{Materials, Energy, Financials}"


In [8]:
df_sector = df.copy()

### match ticker to sector

In [9]:
df_compustat = pd.read_csv('data/compustat_GICS.csv')
df_compustat = df_compustat.sort_values(by=['tic', 'fyear'])
df_compustat = df_compustat.drop_duplicates(subset=['tic'], keep='last')
df_compustat = df_compustat[['tic', 'gsector']]
df_compustat = df_compustat.rename(columns={'tic': 'ticker', 'gsector': 'gics_sector_code'})
df_compustat.reset_index(drop=True, inplace=True)
df_compustat.head()

,ticker,gics_sector_code
0,0015B,25.0
1,0030B,45.0
2,0032A,55.0
3,0033A,55.0
4,0038A,25.0


In [10]:
gics_sector_mapping = {
  10.0: "Energy",
  15.0: "Materials",
  20.0: "Industrials",
  25.0: "Consumer Discretionary",
  30.0: "Consumer Staples",
  35.0: "Health Care",
  40.0: "Financials",
  45.0: "Information Technology",
  50.0: "Communication Services",
  55.0: "Utilities",
  60.0: "Real Estate"
}

df_compustat['gics_sector'] = df_compustat['gics_sector_code'].map(gics_sector_mapping)
df_compustat.dropna(subset=['gics_sector'], inplace=True)
df_compustat.head()

,ticker,gics_sector_code,gics_sector
0,0015B,25.0,Consumer Discretionary
1,0030B,45.0,Information Technology
2,0032A,55.0,Utilities
3,0033A,55.0,Utilities
4,0038A,25.0,Consumer Discretionary


In [11]:
df_sector.head()

,stock_tickers,text_tokens,sectors
newsID,,,
1,[],"[oil, price, could, determine, market, react, ...",{Energy}
2,[],"[zoom, provide, disappointing, revenue, foreca...",{}
3,[],"[wall, street, rally, west, hit, russia, new, ...",{}
4,[],"[weak, manufacturing, drag, gdp, growth, oil, ...","{Industrials, Energy}"
5,[],"[singapore, bank, halt, lend, russian, good, j...","{Materials, Energy, Financials}"


In [12]:
df_compustat.head()

,ticker,gics_sector_code,gics_sector
0,0015B,25.0,Consumer Discretionary
1,0030B,45.0,Information Technology
2,0032A,55.0,Utilities
3,0033A,55.0,Utilities
4,0038A,25.0,Consumer Discretionary


In [13]:
# dictionary mapping ticker to sector
ticker_sector_dict = df_compustat.set_index('ticker')['gics_sector'].to_dict()

# test
ticker_sector_dict.get("AAPL")

'Information Technology'

In [14]:
# Map ticker to GICS sector

def update_sectors_with_ticker_info(row):
  current_sectors = set(row['sectors']) if isinstance(row['sectors'], (set, list, tuple)) else set()
  ticker_list = row['stock_tickers']

  for ticker in ticker_list:
    sector = ticker_sector_dict.get(ticker)
    if sector:
      current_sectors.add(sector)
      
  return current_sectors

df_sector['sectors'] = df_sector.swifter.apply(update_sectors_with_ticker_info, axis=1)

Pandas Apply: 100%|██████████| 1904665/1904665 [00:13<00:00, 139164.69it/s]


In [15]:
df_sector.head()

,stock_tickers,text_tokens,sectors
newsID,,,
1,[],"[oil, price, could, determine, market, react, ...",{Energy}
2,[],"[zoom, provide, disappointing, revenue, foreca...",{}
3,[],"[wall, street, rally, west, hit, russia, new, ...",{}
4,[],"[weak, manufacturing, drag, gdp, growth, oil, ...","{Industrials, Energy}"
5,[],"[singapore, bank, halt, lend, russian, good, j...","{Materials, Energy, Financials}"


In [16]:
df_sector.drop(columns=['text_tokens', 'stock_tickers'], inplace=True)

In [17]:
df_sector.head()

,sectors
newsID,
1,{Energy}
2,{}
3,{}
4,"{Industrials, Energy}"
5,"{Materials, Energy, Financials}"


In [18]:
df_sector.to_parquet('data/sectors.parquet')